In [ ]:
from collections.abc import Iterator

import matplotlib.pyplot as plt
import numpy as np
import torch
from dysts.base import DynSys
from neuralop import H1Loss
from neuralop.models import FNO
from neuralop.training import Trainer
from scipy.integrate import solve_ivp

from panda.utils.plot_utils import apply_custom_style

apply_custom_style("../../config/plotting.yaml")

In [ ]:
class KuramotoShivashinsky(DynSys):
    """Implements the 1+1D KS equation in fourier space"""

    def __init__(self, L: float, modes: int):
        self.ic = None
        super().__init__(metadata_path=None, dimension=2 * modes, parameters={})
        self.L = L
        self.modes = modes
        self.dimension = 2 * self.modes
        self.wave_nums = 2 * np.pi * np.arange(0, self.modes + 2) / self.L
        self.N = self.dimension + 2

        # precompute some quantities
        self.freq_domain = np.zeros(self.modes + 2, dtype=np.complex128)
        self.nonlinear_factor = -0.5 * 1j * self.wave_nums * self.N
        self.diffusion_ffts = self.wave_nums**2 - self.wave_nums**4

    def to_spatial(self, q: np.ndarray, N: int) -> np.ndarray:
        """Inverse FFT of the modes to get u(x) at a certain time

        :param q: array of flattened fourier coefficients (real and imag components), can have batch dimensions
        :param N: grid resolution in the spatial domain

        :returns: solution in the spatial domain
        """
        coeffs = np.zeros(q.shape[:-1] + (self.modes + 2,), dtype=complex)
        coeffs[..., 1:-1] = q[..., : self.modes] + 1j * q[..., self.modes :]
        return np.fft.irfft(coeffs, n=N)

    def rhs(self, t: float, X: np.ndarray) -> np.ndarray:
        self.freq_domain[1:-1] = X[: self.modes] + 1j * X[self.modes :]
        u = np.fft.irfft(self.freq_domain, n=self.N)
        pseudospectral_term = self.nonlinear_factor * np.fft.rfft(u * u)
        linear_term = self.diffusion_ffts * self.freq_domain

        # repackage components
        flow = (linear_term + pseudospectral_term)[1:-1]
        return np.concatenate([np.real(flow), np.imag(flow)])

In [ ]:
ks = KuramotoShivashinsky(L=100, modes=64)

tfinal = 100
rng = np.random.default_rng(12)  # 1234
ic = 0.1 * rng.normal(size=(ks.dimension,))
teval = np.linspace(0, tfinal, 4096)
sol = solve_ivp(ks.rhs, (0, tfinal), ic, method="DOP853", t_eval=teval, rtol=1e-8, atol=1e-8)
ts, freq_traj = sol.t, sol.y.T
spatial_traj = ks.to_spatial(freq_traj, N=ks.dimension)

In [ ]:
grid = np.linspace(0, ks.L, ks.dimension)
plt.figure(figsize=(10, 4))
plt.pcolormesh(ts, grid, spatial_traj.T, cmap="RdBu", shading="gouraud")
plt.colorbar()
plt.ylabel("x")
plt.xlabel("t")
plt.show()

In [ ]:
from dataclasses import dataclass

from panda.utils.data_utils import safe_standardize


@dataclass
class Dataset(torch.utils.data.Dataset):
    traj: torch.Tensor
    num_iterations: int | None = None

    def __len__(self) -> int:
        return self.traj.shape[0] - 1

    def __getitem__(self, idx: int) -> Iterator[tuple[torch.Tensor, torch.Tensor]]:
        x = self.traj[idx]
        y = self.traj[idx + 1]
        return {
            "x": torch.from_numpy(x).unsqueeze(0).float(),
            "y": torch.from_numpy(y).unsqueeze(0).float(),
        }


device_rank = 0
device = f"cuda:{device_rank}"

start_time = 1024
context_length = 512
prediction_length = 256
end_time = start_time + context_length + prediction_length

stand_traj = safe_standardize(spatial_traj, axis=0)
context_window = stand_traj[start_time : start_time + context_length]
prediction_window = stand_traj[start_time + context_length : end_time]

num_epochs = 10000
batch_size = 128

# preprocess training context
train_loader = torch.utils.data.DataLoader(
    Dataset(context_window),
    batch_size=batch_size,
    num_workers=0,
    pin_memory=True,
)
test_loader = torch.utils.data.DataLoader(
    Dataset(prediction_window), batch_size=batch_size, num_workers=0, pin_memory=True
)

operator = FNO(n_modes=(512,), hidden_channels=256, in_channels=1, out_channels=1).to(device)
trainer = Trainer(model=operator, n_epochs=num_epochs, verbose=True, device=device)
optimizer = torch.optim.AdamW(operator.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=context_length // batch_size * num_epochs)
loss_fn = H1Loss(d=1)

trainer.train(
    train_loader=train_loader,
    test_loaders={"eval": test_loader},
    optimizer=optimizer,
    scheduler=scheduler,
    training_loss=loss_fn,
    save_every=2000,
    save_dir="./ks_ckpt",
)

In [ ]:
def plot_forecast(
    ts: np.ndarray,
    grid: np.ndarray,
    trajectory: np.ndarray,
    predictions: np.ndarray,
    run_name: str = "",
    context_length: int = 512,
    save_path: str | None = None,
    v_abs: float | None = None,
    prediction_horizon: int = 128,
    cmap_name: str = "RdBu",
    title_kwargs: dict = {},
    draw_prediction_horizon_line: bool = True,
    figsize: tuple[int, int] = (9, 9),
    show_cbar: bool = True,
    show_ticks: bool = True,
    show_axes_labels: bool = True,
    show_context: bool = True,
):
    fig, axes = plt.subplots(3, 1, sharex=True, figsize=figsize)

    vmin = min(trajectory.min(), predictions.min())
    vmax = max(trajectory.max(), predictions.max())
    vabs = v_abs or max(abs(vmin), abs(vmax))

    if not show_context:
        ts = ts[context_length:]
    for i, (ax, data, label) in enumerate(
        zip(
            axes,
            [trajectory, predictions, predictions - trajectory],
            [
                "Ground Truth",
                f"Predictions ({run_name})" if run_name else "Predictions",
                "Error",
                # f"Prediction Error ({np.mean(np.abs(predictions[context_length:] - trajectory[context_length:])):.2e}) ({run_name})",
            ],
        )
    ):
        if not show_context:
            data = data[context_length:]
        im = ax.pcolormesh(ts, grid, data.T, cmap=cmap_name, shading="gouraud", vmin=-vabs, vmax=vabs)
        if show_ticks:
            ax.set_ylabel("x")
        else:
            ax.set_yticks([])

        ax.set_title(label, **title_kwargs)
        if show_cbar:
            fig.colorbar(im, ax=ax)
        # draw black vertical line at middle of plot (x axis middle)

        if show_context:
            ax.axvline(ts[context_length], color="black", linewidth=1)

        if i == 2 and draw_prediction_horizon_line:
            # draw a black dotted vertical line at the end of 128 pred length window
            start = context_length if show_context else 0
            ax.axvline(
                ts[start + prediction_horizon],
                color="gray",
                linestyle="--",
                linewidth=1,
            )
    if show_ticks:
        axes[-1].set_xlabel("t")
        # axes[-1].set_xticks(ts[::256])
    else:
        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])

    if show_axes_labels:
        axes[-1].set_xlabel("Time", fontweight="bold")
        for ax in axes:
            ax.set_ylabel("x", fontweight="bold")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")

    return vabs


def rollout(ic: torch.Tensor, num_steps: int, model: torch.nn.Module) -> torch.Tensor:
    traj = [ic]
    state = ic.clone()
    for _ in range(num_steps):
        state = model(state)
        traj.append(state)
    return torch.cat(traj, dim=0)


In [ ]:
prediction = (
    rollout(
        torch.from_numpy(context_window[-1]).float().to(device).unsqueeze(0).unsqueeze(0),
        model=operator,
        num_steps=prediction_length,
    )
    .squeeze()[1:]
    .cpu()
    .detach()
    .numpy()
)  # shape: (pred_length, N)

context = spatial_traj[start_time : start_time + context_length]
prediction = safe_standardize(prediction, axis=0, context=context, denormalize=True)
prediction_traj = np.concatenate([context, prediction], axis=0)

_ = plot_forecast(
    ts[start_time:end_time],
    grid,
    spatial_traj[start_time:end_time],
    prediction_traj,
    context_length=context_length,
    cmap_name="RdBu",
    show_context=True,
    figsize=(3.2, 5),
    title_kwargs={"fontweight": "bold", "fontsize": 8},
)
